Install libraries:

1) Install miniconda from the website
2) Add conda to PATH:
     - Start Menu and search for "Environment Variables."
     - Click on "Edit the system environment variables."
     - In the System Properties window, click on "Environment Variables."
     - In the Environment Variables window, find and select the Path.
     - Add new: C:\ProgramData\miniconda3\Scripts (or where yours is located).

3) Install libs:

Create a new virtual environment named 'langchain' with Python 3.12
```
conda create -n langchain python=3.12 -y
```
List all environments (the active one will be marked with an asterisk)
```
conda env list
```
In PowerShell (admin): Set the execution policy to RemoteSigned for the current user
```
Set-ExecutionPolicy RemoteSigned -Scope CurrentUser
```

```
conda activate langchain
pip install langchain langchain_openai langchain_community langgraph ipykernel python-dotenv
```

If you encounter a dependency resolver error with pip:
```
pip install google-auth # Must be <3.0.dev0,>=2.14.1
```

```
ipython kernel install --user --name=langchain
pip install google-search-results
pip install matplotlib
pip install wikipedia
```

In [1]:
import os
import matplotlib.pyplot as plt
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.agent_toolkits.load_tools import get_all_tool_names

In [2]:
# load_tools() is only a shorthand function. It's better to properly use langchain_community.tools
all_tool_names = get_all_tool_names()
print('length: ', len(all_tool_names))

for i in range(0, len(all_tool_names), 10):
    print(" ".join(all_tool_names[i:i+10]))

length:  52
sleep wolfram-alpha google-search google-search-results-json searx-search-results-json bing-search metaphor-search ddg-search google-books google-lens
google-serper google-scholar google-finance google-trends google-jobs google-serper-results-json searchapi searchapi-results-json serpapi dalle-image-generator
twilio searx-search merriam-webster wikipedia arxiv golden-query pubmed human awslambda stackexchange
sceneXplain graphql openweathermap-api dataforseo-api-search dataforseo-api-search-json eleven_labs_text2speech google_cloud_texttospeech read_file reddit_search news-api
tmdb-api podcast-api memorize llm-math open-meteo-api requests requests_get requests_post requests_patch requests_put
requests_delete terminal


Types of LLMs:
- Language models - input stings and generate strings. They are typically older and work best to answer individual user queries.
- Chat model - inputs a sequence of messages, and generates responses that are contextually aware of the conversation flow. By default does not remember past conversations (stateless), but sometimes conversational memory is implemented. 
- Instruct models - optimized to follow specific instructions and perform tasks rather than engage in open-ended dialogue. 

In [3]:
# Chat model

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

chat_model = ChatOpenAI(api_key=api_key, model='gpt-4o-mini')


messages = [SystemMessage(content='You are a Scientist in AI.'),                # instruct a system to behave in certain way: tone, rules, how respond
            HumanMessage(content="Which are the most 3 hot topic in AI now?")]  # input or queries made by the user/human    

print(chat_model.invoke(messages).content)

As of my last update in October 2023, three of the hottest topics in the field of artificial intelligence include:

1. **Generative AI**: This area focuses on models that can generate new content, such as text, images, music, and videos. The rapid development and application of models like OpenAI's GPT-4 and DALL-E have received significant attention, leading to discussions about their ethical implications, use cases, and potential for creative industries.

2. **AI Ethics and Regulation**: The growing capabilities of AI have sparked crucial conversations around ethics, bias, transparency, and accountability. Governments and organizations are exploring frameworks for AI governance to ensure responsible deployment and to mitigate risks associated with AI technologies. This includes discussions on data privacy, algorithmic bias, and the societal impact of AI.

3. **AI in Healthcare**: The use of AI in healthcare continues to expand, particularly in areas such as diagnostics, personalized 

#### Tools are classes that an Agent uses to interact with the world. https://python.langchain.com/api_reference/community/tools.html 

In [4]:
# Google jobs tool

from langchain_community.tools.google_jobs import GoogleJobsQueryRun
from langchain_community.utilities.google_jobs import GoogleJobsAPIWrapper

api_key_serpapi = os.getenv('SERPAPI_API_KEY')
os.environ["SERPAPI_API_KEY"] = api_key_serpapi

# Take special care of description
description_googlejobs = """
A wrapper around Google Jobs Search. 
Useful for when you need to get information aboutgoogle search Jobs from Google JobsInput should be a search query. 
"""

googlejobs = GoogleJobsQueryRun(api_wrapper=GoogleJobsAPIWrapper(), 
                                     description=description_googlejobs)

# Example usage or run(): 
response_g = googlejobs.run("Can I get a list of 3 job posting related to machine learning in alberta")
print(response_g[:1000])


_______________________________________________
Job Title: Machine Learning Resident - Client: Jotson (12 months)
Company Name: Alberta Machine Intelligence Institute
Location: Edmonton, AB, Canada
Description: Salary:

Join us for a unique ML Resident role tackling time-series problems in the energy domain with ML/DL. Youll collaborate with a dynamic and fast-paced team of machine learning scientists and domain experts, developing innovative models and products with energy data.

- Maithrreye Srinivasan, Machine Learning Scientist and Dave Staszak, Lead Machine Learning Scientist

About the Role

This is a paid residency that will be undertaken over a twelve-month period with the potential to be hired by our client afterwards (note: at the discretion of the client and with the requirement of being located in Calgary at that time). The resident will be reporting to an Amii Machine Learning Scientist and regularly consult with the Client team to share insights and engage in knowledge t

In [ ]:
# Golden query tool. 10 free queries
if False: 
    import json
    from langchain_community.utilities.golden_query import GoldenQueryAPIWrapper

    api_key_golden = os.getenv('GOLDEN_API_KEY')
    golden_query = GoldenQueryAPIWrapper()

    json.loads(golden_query.run("companies in nanotech"))

In [5]:
from langchain_community.tools import WikipediaQueryRun  
from langchain_community.utilities import WikipediaAPIWrapper

description_wiki = """
A tool to explain things in text format. Use this tool if you think the user  asked concept is best explained through text.
""" 

wiki_api_wrapper = WikipediaAPIWrapper(doc_content_chars_max=280)
wikipedia = WikipediaQueryRun(description=description_wiki,
                               api_wrapper=wiki_api_wrapper)

In [6]:
tools = [googlejobs, wikipedia]

- This agent (langchain.agents.react.agent.create_react_agent) from LangChain - is not good 
- Better use from the LangGraph (offers a more flexible and full-featured)

There are agent types in LangChain:

- React agents: Use reasoning and actions to decide on the best steps 
- Multi-agent: Divide complicated problems into units of work that can be targeted by specialized agents 
- Conversational agents: Engage in dialogue and maintain context across multiple interactions 
- Structured chat agents: Parse inputs and outputs into structured formats 
- Tool calling agents: Use tools in a straightforward way 
- Self-ask with search: Split queries into smaller steps to handle them 

In [7]:
from langgraph.prebuilt import create_react_agent
 
prompt_system = SystemMessage("You are a recruiter, who is trying to help me in finding suitable jobs")
prompt_human  = HumanMessage("Find me the most recent Machine Learning job in Alberta, specify when the job was posted")

agent = create_react_agent(chat_model,
                           tools,
                           state_modifier=prompt_system)

response = agent.invoke({"messages": prompt_human})

In [ ]:
print(response['messages'][3].content)

The most recent Machine Learning job posted in Alberta is for a **Machine Learning Engineer** at **Ontopical** in Calgary, AB. 

### Job Details:
- **Company:** Ontopical
- **Location:** Calgary, Alberta, Canada
- **Job Description:** The position involves developing, deploying, and maintaining production-ready scalable ML models, among other responsibilities.
- **Qualifications:** M.Sc. in related fields, experience with machine learning packages, cloud platforms, and good organizational skills.

The specific posting date is not mentioned in the provided information, but it is identified as the most recent job in the category. If you want more information or how to apply, please let me know!


In [11]:
response['messages']

[HumanMessage(content='Find me the most recent Machine Learning job in Alberta, specify when the job was posted', additional_kwargs={}, response_metadata={}, id='2bdca494-5473-41d2-8b4f-c0f215dc99a3'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dCpgU0EhVf9X7cnr5H4jrqIx', 'function': {'arguments': '{"query":"Machine Learning jobs in Alberta"}', 'name': 'google_jobs'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 146, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6cad9110-36a5-43a9-870a-6053bc805133-0', tool_calls=[{'name': 'google_jobs', 'args': {'query': 'Machine Learning jobs

In [28]:
response['messages'][1].tool_calls[0]['name']

'google_jobs'

In [13]:
prompt_system = SystemMessage("You are my assistant")
prompt_human  = HumanMessage("What is Amii (research institute)?")

agent = create_react_agent(chat_model,
                           tools,
                           state_modifier=prompt_system)

response1 = agent.invoke({"messages": prompt_human})
print(response1['messages'][3].content)

The Alberta Machine Intelligence Institute (Amii) is a research institute based in Edmonton, Alberta. Established in 2002, its primary focus is on driving innovation in the field of artificial intelligence (AI) and machine learning. Amii collaborates with companies to facilitate the adoption and growth of AI technologies.


In [ ]:
# We now see that it is correctly using wikipedia tool to answer my question

response1['messages'][1].tool_calls[0]['name']

'wikipedia'

In [32]:
response1

{'messages': [HumanMessage(content='What is Amii (research institute)?', additional_kwargs={}, response_metadata={}, id='03937828-ad79-480c-bf3d-f480aeac6adc'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QdZ9ljswbOEVPP9Ja8LbsMRW', 'function': {'arguments': '{"query":"Amii (research institute)"}', 'name': 'wikipedia'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 126, 'total_tokens': 145, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1e06b06d-c8b1-44b0-8d5f-d429e3553698-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Amii (research institute)'}, 'id': 'call_QdZ9ljswbOEVPP9Ja8LbsMRW', 'typ